In [62]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

# Baixe recursos adicionais do NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\higor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\higor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\higor\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [63]:
df = pd.read_csv('data/bolsonaro_tweets.csv')
df.head()

,date,text,likes,retweets,link
0,2020-10-21,- Não se justifica um bilionário aporte financ...,16890,2519,https://twitter.com/jairbolsonaro/status/13189...
1,2020-10-21,A VACINA CHINESA DE JOÃO DORIA\n- Para o meu G...,23349,4174,https://twitter.com/jairbolsonaro/status/13189...
2,2020-10-21,@secomvc @ItamaratyGovBr @MinEconomia @ernesto...,2874,154,https://twitter.com/jairbolsonaro/status/13188...
3,2020-10-21,- Recebi a visita do Conselheiro de Segurança ...,19593,3391,https://twitter.com/jairbolsonaro/status/13188...
4,2020-10-20,"- CURSOS TÉCNICOS. \n- Portaria 524-MEC, de 30...",16221,3227,https://twitter.com/jairbolsonaro/status/13185...


In [64]:
# Verificando se há valores nulos
print(df.isnull().sum())
# Verificando o tipo de dado de cada coluna
print(df.dtypes)
# Descrição do dataset
print(df.shape)

date        7225
text           0
likes          0
retweets       0
link           0
dtype: int64
date        object
text        object
likes        int64
retweets     int64
link        object
dtype: object
(9945, 5)


In [65]:
# Converter a coluna 'date' para o tipo datetime
df['date'] = pd.to_datetime(df['date'])
# Preencher com a data anterior
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['date'] = df['date'].ffill() # forward fill

print(df.isnull().sum())


date        0
text        0
likes       0
retweets    0
link        0
dtype: int64


In [66]:
# Tokenizar os tweets

# Tokenização e pré-processamento
stop_words = set(stopwords.words('portuguese'))
ps = PorterStemmer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    tokens = [ps.stem(token) for token in tokens]
    return tokens

# Aplicar a função de pré-processamento aos tweets
df['tokens'] = df['text'].apply(preprocess_text)

# Visualizar o resultado
print(df[['text', 'tokens']].head())

                                                text  \
0  - Não se justifica um bilionário aporte financ...   
1  A VACINA CHINESA DE JOÃO DORIA\n- Para o meu G...   
2  @secomvc @ItamaratyGovBr @MinEconomia @ernesto...   
3  - Recebi a visita do Conselheiro de Segurança ...   
4  - CURSOS TÉCNICOS. \n- Portaria 524-MEC, de 30...   

                                              tokens  
0  [justifica, bilionário, aport, financeiro, med...  
1  [vacina, chinesa, joão, doria, governo, qualqu...  
2  [secomvc, itamaratygovbr, mineconomia, ernesto...  
3  [recebi, visita, conselheiro, segurança, eua, ...  
4  [curso, técnico, portaria, autoriza, oferta, 5...  


In [67]:
# Análise de Sentimentos
sia = SentimentIntensityAnalyzer()

def analise_sentimento(tokens):
    text = ' '.join(tokens)
    sentimento = sia.polarity_scores(text)
    pont_compost = sentimento['compound']

    if pont_compost >= 0.05:
        return 'Positivo'
    elif pont_compost <= -0.05:
        return 'Negativo'
    else:
        return 'Neutro'

# Aplicar a análise de sentimentos aos tokens
df['sentimento'] = df['tokens'].apply(analise_sentimento)

In [68]:
# Visualizar o resultado
print(df['sentimento'].value_counts(normalize=True))

sentimento
Neutro      0.866365
Positivo    0.076018
Negativo    0.057617
Name: proportion, dtype: float64
